# Adversarial Attacks using API

In [ ]:
from advsecurenet.attacks.gradient_based import FGSM, LOTS
from advsecurenet.shared.types.configs.attack_configs import (
    FgsmAttackConfig,
    LotsAttackConfig,
)
from advsecurenet.shared.types.configs.attack_configs.attacker_config import (
    AttackerConfig,
)
from advsecurenet.models.model_factory import ModelFactory
from advsecurenet.datasets.dataset_factory import DatasetFactory
from advsecurenet.attacks.attacker import Attacker
from advsecurenet.dataloader.data_loader_factory import DataLoaderFactory
from advsecurenet.shared.types.configs.preprocess_config import (
    PreprocessConfig,
    PreprocessStep,
)
from advsecurenet.shared.types.configs.device_config import DeviceConfig
from advsecurenet.utils.adversarial_target_generator import AdversarialTargetGenerator
from advsecurenet.datasets.targeted_adv_dataset import AdversarialDataset

from tqdm.auto import tqdm

import numpy as np

from matplotlib import pyplot as plt

In [ ]:
# Define the model
model = ModelFactory.create_model(
    model_name="resnet18", num_classes=10, pretrained=True
)

In [ ]:
# Lets define the preprocessing configuration we want to use
preprocess_config = PreprocessConfig(
    steps=[
        PreprocessStep(name="Resize", params={"size": 32}),
        PreprocessStep(name="CenterCrop", params={"size": 32}),
        PreprocessStep(name="ToTensor"),
        PreprocessStep(
            name="ToDtype", params={"dtype": "torch.float32", "scale": True}
        ),
        PreprocessStep(
            name="Normalize",
            params={"mean": [0.485, 0.456, 0.406], "std": [0.229, 0.224, 0.225]},
        ),
    ]
)

# Define the dataset
dataset = DatasetFactory.create_dataset(
    dataset_type="cifar10", preprocess_config=preprocess_config, return_loaded=False
)
test_data = dataset.load_dataset(train=False)

In [ ]:
# Define the dataloder
dataloader = DataLoaderFactory.create_dataloader(dataset=test_data, batch_size=32)

In [ ]:
# define the device config
device = DeviceConfig(processor="mps")

# Define the fgsm config
fgsm_config = FgsmAttackConfig(
    targeted=False,
    epsilon=0.1,
    device=device,
)

# Now we can define the attack
attack = FGSM(config=fgsm_config)

## Untargeted FGSM

### Option 1. Using the Attacker

You can use the `Attacker` to run the attack and return the adversarial images in the end. This helps you not to worry about the attack loop and focus on the attack parameters.

In [ ]:
# define the attacker
attacker_config = AttackerConfig(
    model=model,
    attack=attack,
    dataloader=dataloader,
    device=device,
    return_adversarial_images=True,
)

attacker = Attacker(config=attacker_config)

In [ ]:
adv_imgs = attacker.execute()

### Option 2. Manual Iteration
If you prefer to manually run the attack, you can loop through the dataloader to generate the adversarial samples.

In [ ]:
for images, labels in tqdm(dataloader, desc="Attacking"):
    model = model.to(device.processor)
    images, labels = images.to(device.processor), labels.to(device.processor)
    adv_imgs = attack.attack(model, images, labels)
    break

In [ ]:
# Sampling results
bening_pred = model(images).argmax(dim=1)
adv_pred = model(adv_imgs).argmax(dim=1)

cifar10_classes = [
    "airplane",
    "automobile",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
]

index = -1

# find a index that benign prediction is equal to the label but adversarial prediction is not equal to the label
for i in range(len(bening_pred)):
    index += 1
    if bening_pred[index] == labels[index] and adv_pred[index] != labels[index]:
        break

bening_image = images[index].cpu().numpy().transpose(1, 2, 0)
adv_image = adv_imgs[index].cpu().numpy().transpose(1, 2, 0)
diff = np.abs(bening_image - adv_image)

bening_prediction = cifar10_classes[bening_pred[index]]
adv_prediction = cifar10_classes[adv_pred[index]]

plt.figure(figsize=(10, 10))
plt.subplot(1, 3, 1)
plt.imshow(bening_image)
plt.title(f"Original Image: {bening_prediction}")
plt.axis("off")

plt.subplot(1, 3, 2)
plt.imshow(adv_image)
plt.title(f"Adversarial Image: {adv_prediction}")
plt.axis("off")

plt.subplot(1, 3, 3)
plt.imshow(diff)
plt.title("Difference")
plt.axis("off")

plt.show()

## Targeted FGSM

We can also run the targeted FGSM Attack. We will utilize the `AdversarialTargetGenerator` to generate the target labels for the attack.

In [ ]:
# create adversarial target generator
target_generator = AdversarialTargetGenerator()

# Adversarial Target Generator uses indices mapping to generate target labels. By setting overwrite=True, it will overwrite the existing target labels if they exist.
target_labels = target_generator.generate_target_labels(data=test_data, overwrite=True)

# We can create a new dataset with the adversarial target labels. This will be used during the attack.
adv_data = AdversarialDataset(
    base_dataset=test_data,
    target_labels=target_labels,
)

# Since we have a new dataset, we need to create a new dataloader
targeted_dataloader = DataLoaderFactory.create_dataloader(
    dataset=adv_data, batch_size=32
)

targeted_fgsm_config = FgsmAttackConfig(
    targeted=True,
    epsilon=0.1,
    device=device,
)

targeted_fgsm = FGSM(config=targeted_fgsm_config)

## Option 1. Using the Attacker
From this point everything is the same as the untargeted attack. We again have two options to run the attack. Either use the `Attacker` or manually iterate through the dataloader.

In [ ]:
targeted_attacker_config = AttackerConfig(
    model=model,
    attack=targeted_fgsm,
    dataloader=targeted_dataloader,
    device=device,
    return_adversarial_images=True,
)

targeted_fgsm_attacker = Attacker(config=targeted_attacker_config)

# Now we can execute the attack
targeted_adv_imgs = targeted_fgsm_attacker.execute()

### Option 2. Manual Iteration

This step is almost identical to the untargeted attack. The only difference is here we need to loop through the new dataloader that contains the target labels.

In [ ]:
for images, labels, target_labels, _ in tqdm(targeted_dataloader, desc="Attacking"):
    model = model.to(device.processor)
    images, labels, target_labels = (
        images.to(device.processor),
        labels.to(device.processor),
        target_labels.to(device.processor),
    )
    # Note that we are passing the target labels to the attack function
    adv_imgs = attack.attack(model, images, target_labels)

## LOTS Attack

Targeted attacks need target labels to be specified, which is used by the attack to get closer to. `LOTS` attack is a targeted attack, in addition to the target labels it also expects the target images to be specified. With the huge dataset, it is not feasible to specify the target images manually. Therefore, `advsecurenet` provides a way to generate the target images automatically.

In [ ]:
# We need to generate target images and labels for the targeted attack
target_images, target_labels = target_generator.generate_target_images_and_labels(
    data=test_data, overwrite=True
)

# Again lets create a new dataset with the target images and labels
lots_dataset = AdversarialDataset(
    base_dataset=test_data,
    target_images=target_images,
    target_labels=target_labels,
)

# And a new dataloader
lots_dataloader = DataLoaderFactory.create_dataloader(
    dataset=lots_dataset, batch_size=32
)

### Option 1. Using the Attacker


In [ ]:
# LOTS attack expects a deep feature layer to be passed as an argument. We can get the deep feature layer from the model
model.get_layer_names()

In [ ]:
# Lets define the lots config. We pick fc as the deep feature layer
lots_config = LotsAttackConfig(
    device=device, epsilon=0.1, targeted=True, deep_feature_layer="fc"
)

# And create the attack
lots_attack = LOTS(config=lots_config)

In [ ]:
# Lets define the attacker config again
lots_attacker_config = AttackerConfig(
    model=model,
    attack=lots_attack,
    dataloader=lots_dataloader,
    device=device,
    return_adversarial_images=True,
)

lots_attacker = Attacker(config=lots_attacker_config)

# Now we can execute the attack
lots_adv_imgs = lots_attacker.execute()

### Option 2. Manual Iteration

If you prefer to manually run the attack, you can loop through the dataloader to generate the adversarial samples. 


In [ ]:
for images, labels, target_labels, target_images in tqdm(
    lots_dataloader, desc="Attacking"
):
    model = model.to(device.processor)
    images, labels, target_labels, target_images = (
        images.to(device.processor),
        labels.to(device.processor),
        target_labels.to(device.processor),
        target_images.to(device.processor),
    )
    # Note that we are passing the target labels to the attack function
    lots_adv_imgs = lots_attack.attack(model, images, target_labels, target_images)